In [1]:
# ==========================================
# STEP 1: Import Core Libraries
# ==========================================

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#(for later graph processing)
import torch

# Improve numpy printing
np.set_printoptions(suppress=True, precision=4)

print("Libraries loaded successfully.")

Libraries loaded successfully.


In [3]:
# ==========================================
# STEP 2A: Mount Google Drive
# ==========================================

from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted successfully.")

Mounted at /content/drive
Google Drive mounted successfully.


In [4]:
# ==========================================
# STEP 2B: Verify File Exists
# ==========================================

PKL_PATH = "/content/drive/MyDrive/ResPlan.pkl"  

import os
print("File exists:", os.path.exists(PKL_PATH))

File exists: True


In [ ]:
# ==========================================
# STEP 3: Load ResPlan PKL Dataset
# ==========================================

# Replace this with your actual file path
PKL_PATH = "/content/drive/MyDrive/ResPlan.pkl"

import pickle

with open(PKL_PATH, "rb") as f:
    data = pickle.load(f)

print("Dataset loaded successfully.")
print("Type of dataset:", type(data))
print("Number of plans:", len(data))

# Inspect the first sample
sample = data[0]
print("\nKeys in one sample:", sample.keys())

# look at metadata 
if "metadata" in sample:
    print("\nSample metadata:", sample["metadata"])

Dataset loaded successfully.
Type of dataset: <class 'list'>
Number of plans: 17000

Keys in one sample: dict_keys(['balcony', 'bathroom', 'bedroom', 'door', 'garden', 'inner', 'parking', 'pool', 'stair', 'veranda', 'wall', 'window', 'front_door', 'unitType', 'id', 'kitchen', 'land', 'net_area', 'area', 'neighbor', 'living', 'wall_depth', 'storage', 'graph'])


In [7]:
# ==========================================
# STEP 4: Inspect Room Geometry with Shapely
# ==========================================

from shapely.geometry import Polygon, MultiPolygon

room_type = 'bedroom'
room_geom = sample[room_type]

# Check type
print(f"Type of {room_type} geometry:", type(room_geom))

# Handle MultiPolygon vs Polygon
if isinstance(room_geom, MultiPolygon):
    # Pick the first polygon in the multipolygon
    first_polygon = list(room_geom.geoms)[0]
elif isinstance(room_geom, Polygon):
    first_polygon = room_geom
else:
    raise TypeError(f"Unexpected geometry type: {type(room_geom)}")

# Print coordinates of the polygon
print(f"Coordinates of the first {room_type}:")
print(list(first_polygon.exterior.coords))

Type of bedroom geometry: <class 'shapely.geometry.multipolygon.MultiPolygon'>
Coordinates of the first bedroom:
[(255.99999999999994, 88.36565539910274), (255.99999999999994, 19.035401767867448), (177.66374983342956, 19.035401767867448), (177.66374983342956, 88.36565539910274), (255.99999999999994, 88.36565539910274)]


In [8]:
# ==========================================
# STEP 5: Normalize and Center All Floor Plans
# ==========================================

from shapely.geometry import Polygon, MultiPolygon
import numpy as np

def normalize_and_center_plan(plan):
    """
    Normalize and center all room polygons in a floor plan.
    Returns a dictionary with same keys but polygons normalized.
    """
    # Collect all coordinates to compute overall bounding box
    all_coords = []

    # Only normalize geometries (Polygon / MultiPolygon)
    room_keys = [k for k in plan.keys() if isinstance(plan[k], (Polygon, MultiPolygon))]

    for key in room_keys:
        geom = plan[key]
        if isinstance(geom, MultiPolygon):
            for poly in geom.geoms:
                all_coords.extend(list(poly.exterior.coords))
        elif isinstance(geom, Polygon):
            all_coords.extend(list(geom.exterior.coords))

    all_coords = np.array(all_coords)
    min_vals = all_coords.min(axis=0)
    max_vals = all_coords.max(axis=0)
    center = (min_vals + max_vals) / 2
    scale = max(max_vals - min_vals)  # scale by largest side

    # Normalize and center each room
    normalized_plan = {}
    for key in plan.keys():
        geom = plan[key]
        if isinstance(geom, Polygon):
            coords = np.array(geom.exterior.coords)
            coords = (coords - center) / scale
            normalized_plan[key] = Polygon(coords)
        elif isinstance(geom, MultiPolygon):
            normalized_polys = []
            for poly in geom.geoms:
                coords = np.array(poly.exterior.coords)
                coords = (coords - center) / scale
                normalized_polys.append(Polygon(coords))
            normalized_plan[key] = MultiPolygon(normalized_polys)
        else:
            # Keep metadata as-is
            normalized_plan[key] = plan[key]

    return normalized_plan

# Test on first sample
normalized_sample = normalize_and_center_plan(sample)

# Inspect first bedroom after normalization
first_bedroom = list(normalized_sample['bedroom'].geoms)[0] if isinstance(normalized_sample['bedroom'], MultiPolygon) else normalized_sample['bedroom']
print("Coordinates after normalization and centering:")
print(list(first_bedroom.exterior.coords))

Coordinates after normalization and centering:
[(0.46442389232195974, -0.2340651595551164), (0.46442389232195974, -0.4861296324960345), (0.1796162567487296, -0.4861296324960345), (0.1796162567487296, -0.2340651595551164), (0.46442389232195974, -0.2340651595551164)]


In [9]:
# ==========================================
# STEP 6: Preprocess Graph Representation
# ==========================================

import torch
from torch_geometric.data import Data

def preprocess_graph(plan):
    """
    Converts plan['graph'] into PyTorch Geometric Data object
    with normalized node coordinates as features.
    """
    graph = plan['graph']
    nodes = graph['nodes']      # usually a list of dicts with 'id' and 'type'
    edges = graph['edges']      # list of (source, target) tuples

    # Build node features: use room centroids as coordinates
    node_features = []
    for node in nodes:
        room_type = node['type']
        # Get the normalized polygon for this room
        geom = plan.get(room_type)
        if geom is None:
            # fallback zero coords
            centroid = [0.0, 0.0]
        elif isinstance(geom, Polygon):
            centroid = list(geom.centroid.coords)[0]
        elif isinstance(geom, MultiPolygon):
            # use centroid of first polygon
            centroid = list(list(geom.geoms)[0].centroid.coords)[0]
        node_features.append(centroid)

    x = torch.tensor(node_features, dtype=torch.float)   # shape [num_nodes, 2]
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()  # shape [2, num_edges]

    return Data(x=x, edge_index=edge_index)

# Test on normalized sample
graph_data = preprocess_graph(normalized_sample)
print("Graph Data object ready.")
print("Node features (centroids):\n", graph_data.x)
print("Edge index:\n", graph_data.edge_index)

ModuleNotFoundError: No module named 'torch_geometric'